load packages

In [ ]:
import numpy as np
import pandas as pd

from sklearn import svm, model_selection
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

load and split kaggle training data

In [ ]:
df = pd.read_csv('data/train.csv')
#arr = df.values

X = df.loc[:, df.columns != 'label'].as_matrix().astype('uint8')
Y = df.label.as_matrix().tolist()

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.25)

visualize first entry

In [ ]:
plt.imshow(X_train[0].reshape(28,28), cmap=plt.cm.gray_r)
plt.show()

Y_train[0]

train a linear support vector classifier and predict the labels of the test set

In [ ]:
clf = svm.LinearSVC()
clf.fit(X_train, Y_train)

predicted = clf.predict(X_test)
#sorted(clf.cv_results_.keys())

In [ ]:
print("Classification report for classifier %s:\n%s\n" % (clf, metrics.classification_report(Y_test, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(Y_test, predicted))

In [ ]:
#print(svc.coef_)
#print(svc.intercept_)
print("Score: %s" % clf.score(X_test, Y_test))

save the model for later use

In [ ]:
joblib.dump(clf, 'mnist.model.sav')

retrain the model by tuning the parameters

In [ ]:
Cs = np.logspace(-6, -1, 10)
clf = model_selection.GridSearchCV(estimator=svm.LinearSVC(), param_grid=dict(C=Cs), n_jobs=-1)
clf.fit(X_train, Y_train)
print(clf.best_score_)
print(clf.best_estimator_.C)

print("Score: %s" % clf.score(X_test, Y_test))

In [ ]:
predicted = clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n" % (clf, metrics.classification_report(Y_test, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(Y_test, predicted))

save the tuned model

In [ ]:
joblib.dump(clf, 'mnist.tunedmodel.sav')

train a model using random forest classifier

In [ ]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=10,)
clf.fit(X_train,Y_train)

print("Score: %s" % clf.score(X_test, Y_test))

In [ ]:
predicted = clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n" % (clf, metrics.classification_report(Y_test, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(Y_test, predicted))

In [ ]:
joblib.dump(clf, 'mnist.randomforestmodel.sav')

load the trained model and the kaggle sample set then visualize the first entry

In [ ]:
clf2 = joblib.load('mnist.tunedmodel.sav')

df = pd.read_csv('data/test.csv')
X_sample = df.values

In [ ]:
plt.imshow(X_sample[0].reshape(28,28), cmap=plt.cm.gray_r)
plt.show()

predict the labels for the kaggle sample set

In [ ]:
Y_sample = clf2.predict(X_sample)
Y_sample

structure kaggle submission according to requirements

In [ ]:
index = [i for i in range(1, len(Y_sample)+1)]
index
df = pd.DataFrame(Y_sample, index=index, columns=["Label"])
df.index.name = "ImageId"

df.to_csv('data/result.csv')